In [1]:
import rasterio
import numpy as np
from affine import Affine
from pyproj import Proj, transform
import pyproj
# import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import h5py
# import pyhdf
# from pyhdf.SD import SD, SDC
# import re
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
from PIL import Image
import scipy.stats as stats
import numpy.ma as ma

In [2]:
# Using path 121 and row 059

In [5]:
qa_im = Image.open('/Users/nikaankoupaei/Downloads/LE07_L2SP_121059_20130607_20200907_02_T1_QA_PIXEL.TIF')



In [6]:
# Define red, nir, and swir bands
red_im = Image.open('/Users/nikaankoupaei/Downloads/LE07_L2SP_121059_20130607_20200907_02_T1_SR_B3.TIF')
nir_im = Image.open('/Users/nikaankoupaei/Downloads/LE07_L2SP_121059_20130607_20200907_02_T1_SR_B4.TIF')
swir_im = Image.open('/Users/nikaankoupaei/Downloads/LE07_L2SP_121059_20130607_20200907_02_T1_SR_B7.TIF')



In [7]:
# convert to pandas dataframe
red = pd.DataFrame(np.array(red_im))
nir = pd.DataFrame(np.array(nir_im))
swir = pd.DataFrame(np.array(swir_im))

In [6]:
# convert fill value to nans
red=red.replace(0,np.nan)
nir=nir.replace(0,np.nan)
swir=swir.replace(0,np.nan)

In [7]:
# filter through only valid values and replace non-valid values with nans 
mask_red = (red >= 7273) & (red <= 43636) 
mask_nir = (nir >= 7273) & (nir <= 43636)
mask_swir = (swir >= 7273) & (swir <= 43636)

red = red[mask_red]
nir = nir[mask_nir]
swir = swir[mask_swir]

In [8]:
# Multiply by scale factor and add offset
red = (0.0000275*red) - 0.2
nir = (0.0000275*nir) - 0.2
swir = (0.0000275*swir) - 0.2

In [9]:
# DELETE IF NEEDED

red = np.asarray(red)
nir = np.asarray(nir)
swir = np.asarray(swir)

In [10]:
# DELETE IF NEEDED

# slice the dataframes to obtain only region of interest

# rows

# red_arr = np.delete(red,slice(4000,9999999),0)
# nir_arr = np.delete(nir,slice(4000,9999999),0)
# swir_arr = np.delete(swir,slice(4000,9999999),0)

# # columns

# red = np.delete(red_arr,slice(4000,9999999),1)
# nir = np.delete(nir_arr,slice(4000,9999999),1)
# swir = np.delete(swir_arr,slice(4000,9999999),1)


In [11]:
red = red[3000:4000,6600:7600]
nir = nir[3000:4000,6600:7600]
swir = swir[3000:4000,6600:7600]

In [12]:
# check if shape is same for all
print(red.shape)
print(nir.shape)
print(swir.shape)

(1000, 1000)
(1000, 1000)
(1000, 1000)


In [13]:
# Calculate NDVI and assign to variable
ndvi=(nir-red)/(nir+red)

In [14]:
# Delete
ndvi_arr = np.asarray(ndvi) 


In [15]:
# filter ndvi values
ndvi_arr[ndvi_arr < 0] = np.nan
ndvi_arr[ndvi_arr > 1] = np.nan 

In [16]:
ndvi = pd.DataFrame(ndvi_arr)

In [17]:
# convert swir back to array
swir_arr = np.asarray(swir)

In [18]:
# filter swir values
swir_arr[swir_arr < 0.01] = np.nan
swir_arr[swir_arr > 100] = np.nan

In [19]:
swir = pd.DataFrame(swir_arr)

In [20]:
# Calculate STR and assign to variable
str0=((1-swir)**2)/(2*swir)

In [21]:
# THIS IS WHAT TO DO FOR FILTERING SWIR
SWIR=pd.DataFrame(np.asarray(swir).flatten())

In [22]:
SWIR=SWIR.rename(columns={0:'SWIR'})

In [23]:
# END WHAT TO DO FOR FILTERING FOR SWIR

In [24]:
# START WHAT TO DO FOR FILTERING FOR NDVI

In [25]:
NDVI=pd.DataFrame(np.asarray(ndvi).flatten())

In [26]:
NDVI=NDVI.rename(columns={0:'NDVI'})

In [27]:
# (DELETE IF NEEDED)
NDVI['STR'] = ((1-SWIR['SWIR'])**2)/(2*SWIR['SWIR']) 

In [28]:
# MAIN DATAFRAME (DELETE IF NEEDED)
DF = NDVI
DF

,NDVI,STR
0,0.964929,23.308673
1,0.935714,26.587664
2,0.980598,20.752992
3,0.963934,18.684195
4,0.949901,23.308673
...,...,...
999995,NaN,NaN
999996,NaN,NaN
999997,NaN,NaN
999998,NaN,NaN


In [29]:
x_wet=np.linspace(0,1,100)
x_dry=np.linspace(0,1,100)

In [30]:
# convert to array and flatten ndvi and str
ndvi_flat=np.asarray(ndvi).flatten()
str0_flat=np.asarray(str0).flatten()

In [31]:
# convert both to list
ndvi_list = ndvi_flat.tolist()
str0_list = str0_flat.tolist()

In [32]:
# calculate STR max list (change "before" equation below accordingly)
str_max_list=[]
for i in range(len(ndvi_list)):
    if ((30.134*ndvi_list[i])+5.856) > str0_list[i] and ((2.121*ndvi_list[i])-0.296) < str0_list[i]:
            before=(30.134*ndvi_list[i])+5.856
            str_max_list.append(before)
    else:
        str_max_list.append(np.nan) 

In [33]:
# calculate STR min list (change "before" equation below accordingly)
str_min_list=[]
for i in range(len(ndvi_list)):
    if ((30.134*ndvi_list[i])+5.856) > str0_list[i] and ((2.121*ndvi_list[i])-0.296) < str0_list[i]:
            before=(2.121*ndvi_list[i])-0.296
            str_min_list.append(before)
    else:
        str_min_list.append(np.nan)

In [34]:
str0_max_arr = np.asarray(str_max_list)
str0_min_arr = np.asarray(str_min_list)

In [35]:
str_max = str0_max_arr.reshape((red.shape[0], red.shape[1]))
str_min = str0_min_arr.reshape((red.shape[0], red.shape[1]))

In [4]:
qa_arr1 = np.asarray(qa_im)

In [5]:
# NEW CODE

In [6]:
qa_df_NEW_20231222=pd.DataFrame(qa_arr1)

In [7]:
# convert fill value to nans
# qa_df_NEW_20231222=qa_df_NEW_20231222.replace(1,np.nan)
# REMOVED THIS CODE BECAUSE WHEN TRYING TO CONVERT "NAN" TO BINARY THERE WAS A PROBLEM

In [8]:
# filter through only valid values and replace non-valid values with nans 
mask_qa_df_NEW_20231222 = (qa_df_NEW_20231222 >= 5440) & (qa_df_NEW_20231222 <= 16382) | (qa_df_NEW_20231222 == 1)

qa_df_NEW_20231222 = qa_df_NEW_20231222[mask_qa_df_NEW_20231222]

In [9]:
qa_arr=np.asarray(qa_df_NEW_20231222)

In [10]:
# END NEW CODE

In [11]:
qa_arr_sliced = qa_arr[3000:4000,6600:7600]

In [12]:
# DELETE IF NEEDED

# slice the dataframes to obtain only region of interest

# rows

# qa_arr = np.delete(qa,slice(4000,9999999),0)

# columns

# qa_arr = np.delete(qa_arr,slice(4000,9999999),1)


In [13]:
qa_arr_sliced.shape

(1000, 1000)

In [14]:
f_flat = qa_arr_sliced.flatten()

In [15]:
f_flat.shape

(1000000,)

In [16]:
# convert integer array to binary and append to list
binary_int = []
binary_str = []
for i in range(1000000):
    b_array_int1 = int(bin(f_flat[i])[2:])
    b_array_str1 = bin(f_flat[i])[2:]
    binary_int.append(b_array_int1)
    binary_str.append(b_array_str1)

In [17]:
# convert list to array
binary_int = np.asarray(binary_int)
binary_str = np.asarray(binary_str)

In [18]:
# reshape the array
binary_final_int = binary_int.reshape(1000,1000)
binary_final_str = binary_str.reshape(1000,1000)
binary_final_str

array([['1011100001000', '1011100001000', '1011100001000', ..., '1', '1',
        '1'],
       ['1011100001000', '1011100001000', '1011100001000', ..., '1', '1',
        '1'],
       ['1011100001000', '1011100001000', '1011100001000', ..., '1', '1',
        '1'],
       ...,
       ['1', '1', '1', ..., '1', '1', '1'],
       ['1', '1', '1', ..., '1', '1', '1'],
       ['1', '1', '1', ..., '1', '1', '1']], dtype='<U14')

In [19]:
# START TEST

In [20]:
pd.DataFrame(binary_final_str.flatten()).value_counts(dropna=False)

1                 627396
1010101000000     266039
1011100001000      45403
1010110000000      27211
1010101000010       9309
1110100010000       8538
11010100100000      6911
1110100010010       4483
1011001000000       2021
1011001000010       1168
11010100100010       673
1111000010010        398
1111000010000        339
1010110000010        111
dtype: int64

In [20]:
binary_final_str.flatten()[999999]

'1'

In [25]:
binary_final_str.flatten()[0]

'1010101000000'

In [34]:
result = binary_final_str.flatten()[0][-10:-8] == '01'

In [35]:
result

True

In [ ]:
# END TEST

In [51]:
binary_final_str_flat = binary_final_str.flatten()
len(binary_final_str_flat)

1000000

In [30]:
# check if this new array recreates the previous array but in binary 
check_list = []
for i in range(1000):
    for j in range(1000):
        check = binary_final_int[i][j] == int(bin(qa_arr_sliced[i][j])[2:]) 
        check_list.append(check)

In [31]:
# if True, means that statement in the above cell has been fulfilled
for i in range(1000000):
    b = check_list[i] == True  
b

True

In [54]:
# Create a new array where pixels in OPTRAM array 
# with clear skies are kept and every other pixel
# is set to nan

In [55]:
OPTRAM = (str0 - str_min)/(str_max - str_min)

In [56]:
OPTRAM_flat = np.asarray(OPTRAM).flatten()
len(OPTRAM_flat)

1000000

In [57]:
cloud_masked_OPTRAM_list = []
for i in range(1000000):
    if binary_final_str_flat[i][-10:-8] == '01':   
        keep = OPTRAM_flat[i]
        cloud_masked_OPTRAM_list.append(keep)   
    else:
        cloud_masked_OPTRAM_list.append(np.nan)

In [58]:
pd.DataFrame(cloud_masked_OPTRAM_list).value_counts(dropna=False)

NaN         660230
0.522193       216
0.525027       135
0.517108       127
0.475328       126
             ...  
0.311567         1
0.311568         1
0.311569         1
0.311570         1
0.388772         1
Length: 180699, dtype: int64

In [59]:
# BELOW THIS DELETE IF NECESSARY

In [60]:
OPTRAM_FINAL_CLOUD_MASKED = np.asarray(cloud_masked_OPTRAM_list).reshape((1000,1000))
OPTRAM_FINAL_CLOUD_MASKED.shape

(1000, 1000)

In [61]:
pd.DataFrame(OPTRAM_FINAL_CLOUD_MASKED).to_csv('/Users/nikaankoupaei/Downloads/ownCloud/20231222_OPTRAM_FILES_REALIZED_DIDNT_INLCUDE_VALID_RANGE_ENDS_FOR_LANDSAT/MA_UNDRAINED/LANDSAT_7/NEW_FOR_20231222__7_cloud_masked_OPTRAM_20130420_tang_filtered.csv')


